# Scope

this is another take on free form text parsing.

## Limitations

this is a quick and dirty work, and therefore the regular expression could have been made a lot more elegant

## Reference

The following link can be used to test the regular expression https://regex101.com/r/86KIEO/8/

## Author

Gourav Sengupta

If you are interested in solving interesting regular expression based problems please get in touch with [me](g@datamasala.com)

In [1]:
import pandas
import re, regex
import sys, os

In [2]:
pandas.options.display.max_rows = 999
pandas.options.display.max_colwidth = -1

## Environment Details

In [3]:
sys.version

'3.6.2 |Continuum Analytics, Inc.| (default, Jul 20 2017, 13:14:59) \n[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]'

## I. Input

The input is a free form text file which looks like this

```
(LM5678) MU7777 Developer             open order hitrom citron     low
               and		      hdtv ananda bazar paprika.    (TS09)
		Validation					    amit
								   sail

(LM4444) MU8888 Developer             open1 order1 hitromi citroner     medium
               and		      hdtv ananda bazar paprika.    (TS09)
		Validation					    amit
								   saigal

(LM9999) MU9878 Validation	        model framework I am doing. High
					 let it be let it be
								    
					Beatles hdtv hdtv
```

## II. Parsing the result into Pandas Dataframe

### II.a. Entering the text into a variable

this entire text can also be read from a file using the `read()` option and without removing any end of line characters or stripping any end of line strings

In [4]:
textToParse = """(LM5678) MU7777 Developer             open order hitrom citron     low
               and		      hdtv ananda why am I here.    (TS09)
		Validation					    amit
								   sail

(LM4444) MU8888 Developer             open1 order1 hitromi citroner     medium
               and		      hdtv ananda bazar paprika.    (TS09)
		Validation					    amit
								   saigal

(LM9999) MU9878 Validation	        model framework I am doing. High
					 let it be let it be
								    
					Beatles hdtv hdtv

(LM99) MU9878 Testing	        model framework I am doing. Low
					 let it be let it be
								    
					Beatles things make hdtv
                    difference
                    
                    
                    expertise
(LM9599) MU9878 Validation	        Testing other variations. medium
					 let it be let it be
								    
					Beatles hdtv hdtv
                    yuyuiuyi
                    



            testing                          rrr                         TestING"""

### II.b. Creating the Pandas Dataframe

In [470]:
def createPandasDataFrame(textToParse):
    regularExpr  = "(\s*)?(?<field1>\(LM\d+\))\s(?<field2>MU\d+)\s(?<field3>[a-zA-Z]+)\s+(?<field4part1>([a-zA-Z0-9\.]+\s)+)(\s+)?(?<field5>[lL]ow|[mM]edium|[hH]igh)([\s|\n]+)?(?<field4part2>(((?!\(LM\d+\)).)*[\s|\n]?)+)"
    parsedResult = regex.findall(regularExpr, textToParse)
    collatedResult = map(lambda record: (record[1],
                                         record[2],
                                         record[3],
                                         record[7],
                                         re.sub("\s+|\t+", " ", re.sub("\r|\n", "", record[4] + record[9]))), 
                         parsedResult)
    return(collatedResult)

## III. Showing the result

In [477]:
pandas.DataFrame(list(createPandasDataFrame(textToParse)), 
                 columns = ["field1", "field2", "field3", "field4", "field5"])

,field1,field2,field3,field4,field5
0,(LM5678),MU7777,Developer,low,open order hitrom citron and hdtv ananda why am I here. (TS09) Validation amit sail
1,(LM4444),MU8888,Developer,medium,open1 order1 hitromi citroner and hdtv ananda bazar paprika. (TS09) Validation amit saigal
2,(LM9999),MU9878,Validation,High,model framework I am doing. let it be let it be Beatles hdtv hdtv
3,(LM99),MU9878,Testing,Low,model framework I am doing. let it be let it be Beatles things make hdtv difference expertise
4,(LM9599),MU9878,Validation,medium,Testing other variations. let it be let it be Beatles hdtv hdtv yuyuiuyi testing rrr TestING


## Comments

The regular expression has now named groups, and can therefore be studied quite easily. There is no documentation provided as of now on how to read the regular expression


## Input 



```
(LM5678) MU7777 Developer             open order hitrom citron     low
               and		      hdtv ananda why am I here.    (TS09)
		Validation					    amit
								   sail

(LM4444) MU8888 Developer             open1 order1 hitromi citroner     medium
               and		      hdtv ananda bazar paprika.    (TS09)
		Validation					    amit
								   saigal
```


Now let us say that the above data has to be parsed and entered as the following:

| field1 	| field2 	| field3 	| field4 	| field5                  	|
|--------	|--------	|--------	|--------	|-------------------------	|
| (LM5678) 	| MU7777 	| Developer and Validation 	| open order hitrom citron  hdtv ananda why am I here. (TS09) amit sail	| low 	|

In [187]:
#note this is used to identify all the words that will make up the third column, in case you do not want a word like 
#and, this can be excluded separately, this has to be case sensitive as well.
allowedValuesInThirdField = "[dD]eveloper|and|[vV]alidation"

#note this is used to identify all the words that will make up the fifth column, in case you do not want a word like 
#and, this can be excluded separately, this has to be case sensitive as well.
allowedValuesInFifthField = "[lL]ow|[mM]edium|[hH]igh"

#the values of columns after the first row containing (LM****) are separated by multiple spaces, this is for 
# identifying the minimum number of spaces that are separating the column. If the number of spaces is mentioned to be
# 3, then the parser will look for multiples of 3. 
spacesBetweenColumnsFromNextRow = 5

regularExpr = "(\s*)?(?<field1>\(LM\d+\))\s(?<field2>MU\d+)\s(?<field3>" + allowedValuesInThirdField + "+)\s+(?<field4part1>([a-zA-Z0-9\.]+\s)+)(\s+)?(?<field5>" + allowedValuesInFifthField + ")(?<nextrows>(([\s|\n]?(?!\(LM\d+\)).)*)+)"
parsedRecords = regex.findall(regularExpr, textToParse)

In [179]:
for record in parsedRecords:
    firstField  = record[1]
    secondField = record[2]
    thirdField  = record[3]
    fourthField = record[4]
    fifthField  = record[7]
    nextRow = list(filter(lambda value: value != '', re.split(string = record[8], pattern = "[\t|\s]{" + str(spacesBetweenColumnsFromNextRow) + "}")))
    print(nextRow)

[' and', '   hdtv ananda why am I here.    (TS09)\n\t\tValidation', '    amit', '  sail']
[' and', '   hdtv ananda bazar paprika.    (TS09)\n\t\tValidation', '    amit', '  saigal']
['\t let it be let it be', '\t\t\t\tBeatles hdtv hdtv']
['\t let it be let it be', '\t\t\t\tBeatles hdtv hdtv', ' yuyuiuyi', ' ']


In [180]:
nextRow = parsedRecords[0][8]
print(nextRow)


               and		      hdtv ananda why am I here.    (TS09)
		Validation					    amit
								   sail


remove __NULL__ values

In [181]:
list(filter(lambda value: value != '', re.split(pattern = "\n+", string = fieldCheck)))

['               and\t\t      hdtv ananda why am I here.    (TS09)',
 '\t\tValidation\t\t\t\t\t    amit',
 '\t\t\t\t\t\t\t\t   sail']

In [182]:
splitRowValues = list(filter(lambda value: value != '', re.split(pattern = "\n+", string = fieldCheck)))

split values between rows

In [183]:
splitValuesInRows = list(map(lambda rowValue: re.split("[\s|\t]{" + str(spacesBetweenColumnsFromNextRow) + "}", rowValue), splitRowValues))

In [184]:
print(splitValuesInRows)

[['', '', '', 'and', '   hdtv ananda why am I here.    (TS09)'], ['\t\tValidation', '    amit'], ['', '', ' sail']]


remove __NULL__ values in the split above

In [185]:
splitValuesInRows = list(map(lambda rowValue: list(filter(lambda value: value != ""
                                                          ,re.split("[\s|\t]{" + str(spacesBetweenColumnsFromNextRow) + "}", rowValue)))
                             , splitRowValues))

In [186]:
print(list(splitValuesInRows))

[['and', '   hdtv ananda why am I here.    (TS09)'], ['\t\tValidation', '    amit'], [' sail']]


iterate through the values in rows and check whether the first string belongs to the __allowedValuesInThirdField__ then allocate it to third field else to the fifth field

In [254]:
thirdFieldValue = parsedRecords[0][3]
print(thirdFieldValue)

Developer


In [255]:
fourthFieldValue = parsedRecords[0][4]
print(fourthFieldValue)

open order hitrom citron 


In [256]:
for h in splitValuesInRows:
    #does first value belong to the third field?
    if (re.match(allowedValuesInThirdField, h[0].strip())):
        thirdFieldValue += " " + h[0].strip()
    else:
        fourthFieldValue = fourthFieldValue.strip() + " " + h[0].strip()
    #append all other values to the fourth field
    if (len(h) > 1):
        fourthFieldValue = fourthFieldValue.strip() + " " + " ".join(list(map(lambda value: value.strip(), h[1:])))

In [258]:
print(thirdFieldValue)

Developer and Validation


In [259]:
print(fourthFieldValue)

open order hitrom citron hdtv ananda why am I here.    (TS09) amit sail
